#Capstone Project - Whats in Shanghai?

###*Lawrence Sheed*

---

In [2]:
#Disable pip's bloody annoying warnings
%env _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1 

#disable python warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#install libraries needed
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install geopy
!pip install geocoder

#import libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json
import folium 
from bs4 import BeautifulSoup  

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    

from sklearn.cluster import KMeans
from IPython.display import display_html
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import geocoder # import geocoder

print('Libraries imported.')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



env: _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1
Libraries imported.


In [1]:
#Foursquare login BS

CLIENT_ID = 'MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK' # your Foursquare ID
CLIENT_SECRET = 'COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH' # your Foursquare Secret
#google access code DB05TYWDA0F1EPAJFETS52ZMQ3PWDDYFXNVAD345H51QLTYF#_=_
ACCESS_TOKEN = '0JXEDHYX0LFM1M3P1XLVEV0TNGWOGPVCE4PKVR3E4LSX2MWA' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK
CLIENT_SECRET:COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH


##Get Location for Fuxing Road in Shanghai, Xu Hui District

In [5]:
address = '1331 Fuxing Zhong Lu, Shanghai'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

#Completely innaccurate, sadly
print ('FourSquare is _wrong_ *CORRECT RESULTS ARE* : 31.211398,121.453289')
#Actual Longtitude / Lat is
# 31.211398,121.453289 
latitude = 31.211398
longtitude = 121.453289 



31.2238233 121.4858025
FourSquare is wrong *CORRECT RESULTS ARE* : 31.211398,121.453289


##Get restaurants in the *correct* area

##Search for restaurants in the area

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

In [75]:
search_query = 'Restaurant'
radius = 5000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [76]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK&client_secret=COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH&ll=31.234354815972843,121.47875408149348&oauth_token=0JXEDHYX0LFM1M3P1XLVEV0TNGWOGPVCE4PKVR3E4LSX2MWA&v=20180604&query=Restaurant&radius=5000&limit=30'

In [78]:
results = requests.get(url).json()
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,location.neighborhood,venuePage.id
0,4cbd9032035d236a0d79eb4e,Fukui Japanese Restaurant (福井日本料理),"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1627393920,False,339 Guangdong Rd,Shandong M Rd,31.234410,121.479948,"[{'label': 'display', 'lat': 31.23441001961838...",113,CN,Huangpu,上海市,中国,"[339 Guangdong Rd (Shandong M Rd), Huangpu, 上海市]",NaN,NaN,NaN
1,4cde7f61c4f6a35d802bca6c,Charme Restaurant (港丽餐厅),"[{'id': '52af3a7c3cf9994f4e043bed', 'name': 'C...",v-1627393920,False,268 Xizang Rd | 西藏中路268号,6/F Raffles City | 来福士广场,31.234173,121.472205,"[{'label': 'display', 'lat': 31.23417308379205...",623,CN,Huangpu,上海市,中国,[268 Xizang Rd | 西藏中路268号 (6/F Raffles City | ...,200085,People's SQ,NaN
2,4ef0130a775b121472c58f97,Three Street Restaurant (三玛璐),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1627393920,False,黄浦区汉口路413号,NaN,31.237118,121.477995,"[{'label': 'display', 'lat': 31.23711818329258...",315,CN,Huangpu,上海市,中国,"[黄浦区汉口路413号, Huangpu, 上海市]",NaN,NaN,NaN
3,4b8b4d8ff964a520869b32e3,Lost Heaven Yunnan Restaurant (花马天堂云南餐厅),"[{'id': '52af3b9a3cf9994f4e043c07', 'name': 'Y...",v-1627393920,False,17a Yan'an E Rd,NaN,31.234711,121.486311,"[{'label': 'display', 'lat': 31.23471124611373...",720,CN,Huangpu,上海市,中国,"[17a Yan'an E Rd, Huangpu, 上海市, 200002]",200002,NaN,NaN
4,4ba35b98f964a520223738e3,Xinwang Restaurant (新旺茶餐厅),"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1627393920,False,汉口路309号 | 309 Hankou Rd.,山东中路 | Shandong Rd. (M),31.237748,121.479868,"[{'label': 'display', 'lat': 31.23774752578747...",392,CN,Huangpu,上海市,中国,[汉口路309号 | 309 Hankou Rd. (山东中路 | Shandong Rd....,NaN,NaN,NaN


In [79]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

print ("Show the Panda Data Store for data with Venue Names and location data")
dataframe_filtered.head()

Show the Panda Data Store for data with Venue Names and location data


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id
0,Fukui Japanese Restaurant (福井日本料理),Sushi Restaurant,339 Guangdong Rd,Shandong M Rd,31.234410,121.479948,"[{'label': 'display', 'lat': 31.23441001961838...",113,CN,Huangpu,上海市,中国,"[339 Guangdong Rd (Shandong M Rd), Huangpu, 上海市]",NaN,NaN,4cbd9032035d236a0d79eb4e
1,Charme Restaurant (港丽餐厅),Cantonese Restaurant,268 Xizang Rd | 西藏中路268号,6/F Raffles City | 来福士广场,31.234173,121.472205,"[{'label': 'display', 'lat': 31.23417308379205...",623,CN,Huangpu,上海市,中国,[268 Xizang Rd | 西藏中路268号 (6/F Raffles City | ...,200085,People's SQ,4cde7f61c4f6a35d802bca6c
2,Three Street Restaurant (三玛璐),Chinese Restaurant,黄浦区汉口路413号,NaN,31.237118,121.477995,"[{'label': 'display', 'lat': 31.23711818329258...",315,CN,Huangpu,上海市,中国,"[黄浦区汉口路413号, Huangpu, 上海市]",NaN,NaN,4ef0130a775b121472c58f97
3,Lost Heaven Yunnan Restaurant (花马天堂云南餐厅),Yunnan Restaurant,17a Yan'an E Rd,NaN,31.234711,121.486311,"[{'label': 'display', 'lat': 31.23471124611373...",720,CN,Huangpu,上海市,中国,"[17a Yan'an E Rd, Huangpu, 上海市, 200002]",200002,NaN,4b8b4d8ff964a520869b32e3
4,Xinwang Restaurant (新旺茶餐厅),Chinese Restaurant,汉口路309号 | 309 Hankou Rd.,山东中路 | Shandong Rd. (M),31.237748,121.479868,"[{'label': 'display', 'lat': 31.23774752578747...",392,CN,Huangpu,上海市,中国,[汉口路309号 | 309 Hankou Rd. (山东中路 | Shandong Rd....,NaN,NaN,4ba35b98f964a520223738e3


## Show the names of the venues

In [80]:
newdf = dataframe_filtered[['name','id']]
newdf


,name,id
0,Fukui Japanese Restaurant (福井日本料理),4cbd9032035d236a0d79eb4e
1,Charme Restaurant (港丽餐厅),4cde7f61c4f6a35d802bca6c
2,Three Street Restaurant (三玛璐),4ef0130a775b121472c58f97
3,Lost Heaven Yunnan Restaurant (花马天堂云南餐厅),4b8b4d8ff964a520869b32e3
4,Xinwang Restaurant (新旺茶餐厅),4ba35b98f964a520223738e3
5,Waya Restaurant (蓝方餐厅),57162eb8cd10c0b025eb22fb
6,Charme Restaurant (港丽餐厅),4b9880cef964a520f54435e3
7,TAI HING Restaurant (太兴餐厅),4e86eb2c722e2f21dede6a3c
8,Qian Restaurant (黔香阁),5b0a25db418686002c623488
9,ROOF 325 Rooftop Restaurant & Bar,4b0588c1f964a52020d822e3


In [81]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the  复兴中路

# add the FourSquare Data as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

##Get the details for Saizayeri (Japanese Chain Italian food)

In [82]:
venue_id = '4daa62378154abafc2aca941' # ID of Tsui Wah
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4daa62378154abafc2aca941?client_id=MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK&client_secret=COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH&oauth_token=0JXEDHYX0LFM1M3P1XLVEV0TNGWOGPVCE4PKVR3E4LSX2MWA&v=20180604'

In [83]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'like', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '4daa62378154abafc2aca941',
 'name': 'Tsui Wah Restaurant (翠华餐厅)',
 'contact': {'phone': '02133662636', 'formattedPhone': '021 3366 2636'},
 'location': {'address': '西藏南路251号宏基西门厅106室 | 251 Xizang Rd. S.',
  'crossStreet': '近淮海东路 | near Huai Hai Rd. E.',
  'lat': 31.226480082225272,
  'lng': 121.4762296741061,
  'labeledLatLngs': [{'label': 'display',
    'lat': 31.226480082225272,
    'lng': 121.4762296741061}],
  'postalCode': '200021',
  'cc': 'CN',
  'city': '上海市',
  'state': '上海市',
  'country': '中国',
  'formattedAddress': ['西藏南路251号宏基西门厅106室 | 251 Xizang Rd. S. (近淮海东路 | near Huai Hai Rd. E.)',
   '上海市',
   '上海市, 200021']},
 'canonicalUrl': 'https://foursquare.com/v/%E7%BF%A0%E5%8D%8E%E9%A4%90%E5%8E%85--tsui-wah-restaurant/4daa62378154abafc2aca941',
 'categories': [{'id': '52af3add3cf9994f4e043bf5',
   'name': 'Hong Kong Restaurant',
   'pluralName': 'Hong Kong Restaurants',
   'shortName': 'Hong Kong',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian

## Get the Venue Rating

In [89]:
try:
    print(result['response']['venue']['name'])
    print("Rating: ",result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')


print ("Lat / Long: ")
print(result['response']['venue']['location']['lat'])
print(result['response']['venue']['location']['lng'])

latitude = result['response']['venue']['location']['lat']
longitude = result['response']['venue']['location']['lng']
name = result['response']['venue']['name']

Tsui Wah Restaurant (翠华餐厅)
Rating:  5.6
Lat / Long: 
31.226480082225272
121.4762296741061


##Lets explore around this location


In [90]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=MGHGZ0OLVSC4UT1XM4Z5IEH0UKRQ5YOT5XZ42SLUWXDDPUEK&client_secret=COFCZNWCRD1HKCRIJ5B3M1XP1TLQ04HEVPJASFWO1PGILHSH&ll=31.226480082225272,121.4762296741061&v=20180604&radius=5000&limit=30'

In [91]:
import requests

results = requests.get(url).json()
'There are {} results around {} restaurant.'.format(len(results['response']['groups'][0]['items']),name)

'There are 30 results around Tsui Wah Restaurant (翠华餐厅) restaurant.'

In [92]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4dc904f745dd9aa940b422a6',
  'name': 'Fraser Residence Shanghai',
  'location': {'address': '98 Shouning Rd | 寿宁路98号',
   'lat': 31.225639286106947,
   'lng': 121.47617694700448,
   'labeledLatLngs': [{'label': 'display',
     'lat': 31.225639286106947,
     'lng': 121.47617694700448}],
   'distance': 93,
   'postalCode': '200021',
   'cc': 'CN',
   'city': 'Huangpu',
   'state': '上海市',
   'country': '中国',
   'formattedAddress': ['98 Shouning Rd | 寿宁路98号',
    'Huangpu',
    '上海市, 200021',
    '中国']},
  'categories': [{'id': '4bf58dd8d48988d1fa931735',
    'name': 'Hotel',
    'pluralName': 'Hotels',
    'shortName': 'Hotel',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4dc904f745dd9aa94

##Convert those to Dataframe

In [93]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Fraser Residence Shanghai,Hotel,98 Shouning Rd | 寿宁路98号,31.225639,121.476177,"[{'label': 'display', 'lat': 31.22563928610694...",93,200021,CN,Huangpu,上海市,中国,"[98 Shouning Rd | 寿宁路98号, Huangpu, 上海市, 200021...",NaN,NaN,4dc904f745dd9aa940b422a6
1,Green Massage (青籁养身),Massage Studio,58 Taicang Rd | 太仓路58号,31.223859,121.472873,"[{'label': 'display', 'lat': 31.22385936320260...",432,NaN,CN,Huangpu,上海市,中国,"[58 Taicang Rd | 太仓路58号 (at Jinan Rd), Huangpu...",at Jinan Rd,NaN,4bbb03d83db7b713b3d8239a
2,Shanghai Concert Hall (上海音乐厅),Concert Hall,523 Yan'an E Rd | 延安东路523号,31.228842,121.473689,"[{'label': 'display', 'lat': 31.22884238131837...",357,200021,CN,Huangpu,上海市,中国,"[523 Yan'an E Rd | 延安东路523号, Huangpu, 上海市, 200...",NaN,NaN,4b0588c3f964a520d0d822e3
3,"Andaz Xintiandi, Shanghai (上海新天地安达仕酒店)",Hotel,88 Songshan Rd,31.221941,121.475431,"[{'label': 'display', 'lat': 31.221941, 'lng':...",510,200021,CN,Huangpu,上海市,中国,"[88 Songshan Rd, Xīntiāndì, Huangpu, 上海市, 2000...",NaN,Xīntiāndì,4e1c179245dd10c6b719a695
4,Campanile Hotel and Restaurant,Hotel,33 Fujian S Rd,31.232123,121.479144,"[{'label': 'display', 'lat': 31.23212310610490...",686,NaN,CN,Huangpu,上海市,中国,"[33 Fujian S Rd, Huangpu, 上海市, 中国]",NaN,NaN,5801b06138facdba59353259
5,Green Massage (青籁养生),Spa,300 Huaihai M Rd | 淮海中路300号,31.225242,121.469261,"[{'label': 'display', 'lat': 31.22524167218201...",677,200021,CN,Huangpu,上海市,中国,[300 Huaihai M Rd | 淮海中路300号 (3/F-05 K11 Art M...,3/F-05 K11 Art Mall | 上海K11购物艺术中心,Huáihǎi Zhōnglù,5193717b498e93db9d73bbcc
6,The Langham Xintiandi (上海新天地朗廷酒店),Hotel,"新天地马当路99号, 黄浦区",31.223197,121.469835,"[{'label': 'display', 'lat': 31.22319682178447...",710,200021,CN,Huangpu,上海市,中国,"[新天地马当路99号, 黄浦区 (Taicang Road, Opposite Xintia...","Taicang Road, Opposite Xintiandi",Xīntiāndì,4e7c6f809a52ecfa7b3ff407
7,Yu Garden Café,Café,"Lobby, Renaissance Hotel",31.229482,121.483501,"[{'label': 'display', 'lat': 31.22948162916867...",768,NaN,CN,Huangpu,上海市,中国,"[Lobby, Renaissance Hotel, Huangpu, 上海市, 中国]",NaN,NaN,4de6e62ad164402a0f1cc92f
8,Ps Café,Café,Huai Hai Zhong Lu 333,31.224881,121.469708,"[{'label': 'display', 'lat': 31.224881, 'lng':...",645,NaN,CN,上海市,上海市,中国,"[Huai Hai Zhong Lu 333, 上海市, 上海市, 中国]",NaN,NaN,5c40799be17910002cb23ed9
9,Polux by Paul Pairet,French Restaurant,Lane 181 Taicang Rd | 太仓路181弄,31.223098,121.470509,"[{'label': 'display', 'lat': 31.22309845942651...",661,NaN,CN,Huangpu,上海市,中国,[Lane 181 Taicang Rd | 太仓路181弄 (5 Xintiandi N ...,5 Xintiandi N | 新天地北里5号,NaN,5c922188fd16bb002c5abbfa


In [94]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup=name,
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

##Whats Trending Closeby

In [95]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6100107ce44e9f3a3b95e82e'},
 'response': {'venues': []}}

In [102]:
print ("Results: ", results['response']['venues'])
print ("Len: " , len(results['response']['venues']))

if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    print (trending_venues_df)
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

    trending_venues_df.head()



Results:  []
Len:  0
No trending venues are available at the moment!
